In [4]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
from unidecode import unidecode
import numpy as np
from itertools import chain


#make sure do not go over rate limit of 20 requests per minute. If over, sportsreference puts in "jail" for an hour
rate = 0
def checkRate(r):
    global rate
    time.sleep(5)

#since roster table is hidden in comments, needs this to fish it out
def makeCommentTable(soup1, type):
    global rate

    #finds all comments in soup and makes them not comments and just html. 
    comments = soup1.find_all(string=lambda text: isinstance(text, Comment) and "table_container" in text)
    html = str(comments).replace('<!--', '').replace('-->', '')
    
    #parse through the HTML content using Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    #reads roster table html into a dataframe
    if type == False:
        #find the roster table by its HTML id
        rosterTable = soup.find('table', {'id': 'roster'})
        df = pd.read_html(str(rosterTable))[0]
    else:
        passingTable = soup.find('table', {'id': 'passing'})
        otherTable = soup.find('table', {'id': 'rushing_and_receiving'})

        #make array of two df instead of one, then flatten it to 1d array to be used later
        df = [pd.read_html(str(passingTable)), pd.read_html(str(otherTable))]
        df = list(chain.from_iterable(df))

    return df

def rosterMaker():

    #empty df to add things to
    df = pd.DataFrame(columns= ["No.", "Player", "Age", "Pos", "G", "GS", "Wt", "Ht", "College/Univ", "BirthDate", "Yrs", "AV", "Drafted (tm/rnd/yr)", "Year", "YearsBack", "Team"])

    global rate

    #years used for grading. changes each season. if making currYearRoster, use array below with just current year
    #years = ["2022", "2021", "2020", "2019"]
    years = ["2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023"]

    #team abbr list
    teams = ["crd", "atl", "rav", "buf", "car", "chi", "cin", "cle", "dal", "den", "det", "gnb", "htx", "clt", "jax", "kan", "rai", "sdg", "ram", "mia", "min", "nwe", "nor", "nyg", "nyj", "phi", "pit", "sfo", "sea", "tam", "oti", "was"]

    #used to find yearsback from present
    x = 1

    #loop to go through each team in each year and make df of roster for each team per year
    for num in years:
        for item in teams: 
            #makes url for every team
            url = "https://www.pro-football-reference.com/teams/" + item + "/" + num + "_roster.htm"

            #get page wanted and make a beautiful soup out of it.
            checkRate(rate)
            response = requests.get(url)
            print(response)
            rate += 1

            soup = BeautifulSoup(response.text, 'html.parser')

            #dataframe of table
            table = makeCommentTable(soup, False)

            #add in year and years back columns
            table['Year'] = num
            table["YearsBack"] = x
            table["Team"] = item


            #make all into one df
            df = pd.concat([df, table], ignore_index=True, join="inner")
                
        x+=1
        
        
    
    #write dfs into csv files for later use
    if len(years)==1:
        #change all rb type positions to rb
        rbValues = ['RB', 'HB', 'TB', 'FB', "LH", "RH", "BB", "B", "WB"]
        df.loc[df['Pos'].isin(rbValues), 'Pos'] = "RB"

        #make only first two strings kept, and only keep rb, wr, qb, te columns
        df.loc[df['Pos'].str.len() > 2, 'Pos'] = df.loc[df['Pos'].str.len() > 2, 'Pos'].apply(lambda x: x[:2])
        keepers = ["RB", "QB", "TE", "WR"]

        df = df.loc[df['Pos'].isin(keepers)]

        df.to_csv("currYearRoster.csv", encoding='utf-8', index=False)
    else:
        df.to_csv("teamsPastRoster.csv", encoding='utf-8', index=False)

rosterMaker()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [4]:
import pandas as pd
teamsPast = pd.read_csv("teamsPastRoster.csv")
teamsPast['YearsBack'] = 2024 - teamsPast['Year'].astype(int)
teamsPast.to_csv('/Users/kmaran3/Dropbox/Darkhorse/teamsPastRoster.csv')